# 测试Lexer

In [1]:
from cilly.lexer.scanner import *
from cilly.lexer.tokens import *
from cilly.exceptions import *
from cilly.parser.parser import *

In [5]:
"""测试空输入"""
tokenize("")

[['eof', None]]

In [6]:
"""测试空白字符处理"""
tokenize("   \t\n\r  ")

[['eof', None]]

In [7]:
"""测试数字识别"""
tokenize("42")
tokenize("3.14")
tokenize("42 3.14")

[['num', 42], ['num', 3.14], ['eof', None]]

In [8]:
"""测试字符串识别"""
tokenize('"Hello, World!"')

[['str', 'Hello, World!'], ['eof', None]]

In [9]:
"""测试标识符和关键字识别"""
tokenize("variable_name")

[['id', 'variable_name'], ['eof', None]]

In [10]:
tokenize("var if else while")

[['var', None], ['if', None], ['else', None], ['while', None], ['eof', None]]

In [ ]:
"""测试运算符识别"""
tokenize("+ - * / ( )")

[['+', None],
 ['-', None],
 ['*', None],
 ['/', None],
 ['(', None],
 [')', None],
 ['eof', None]]

In [12]:
tokenize(">= <= == != && ||")

[['>=', None],
 ['<=', None],
 ['==', None],
 ['!=', None],
 ['&&', None],
 ['||', None],
 ['eof', None]]

In [13]:
"""测试复杂表达式"""
source = 'var x = 42 + y * 3.14; if (x >= 100) { print("Large!"); }'
tokenize(source)

[['var', None],
 ['id', 'x'],
 ['=', None],
 ['num', 42],
 ['+', None],
 ['id', 'y'],
 ['*', None],
 ['num', 3.14],
 [';', None],
 ['if', None],
 ['(', None],
 ['id', 'x'],
 ['>=', None],
 ['num', 100],
 [')', None],
 ['{', None],
 ['print', None],
 ['(', None],
 ['str', 'Large!'],
 [')', None],
 [';', None],
 ['}', None],
 ['eof', None]]

In [2]:
tokens = """
var x = 10; // 这是一个注释
x = x + 1;  // 增加x的值
"""
tokenize(tokens)

[['var', None],
 ['id', 'x'],
 ['=', None],
 ['num', 10],
 [';', None],
 ['id', 'x'],
 ['=', None],
 ['id', 'x'],
 ['+', None],
 ['num', 1],
 [';', None],
 ['eof', None]]

In [3]:
tokens = """
//123
"""
tokenize(tokens)

[['eof', None]]

In [14]:
"""测试无效字符处理"""
tokenize("@")

LexerError: cilly lexer : 非法字符: @

# 测试Parser

In [ ]:
"""测试空程序"""
tokens = tokenize("")
parse(tokens)

['program', []]

In [ ]:
"""测试变量定义"""
tokens = tokenize("var x = 42;")
parse(tokens)

['program', [['define', 'x', ['num', 42]]]]

In [ ]:
"""测试赋值语句"""
tokens = tokenize("x = 42;")
parse(tokens)

['program', [['assign', 'x', ['num', 42]]]]

In [ ]:
"""测试打印语句"""
tokens = tokenize('print("Hello, World!");')
parse(tokens)

['program', [['print', [['str', 'Hello, World!']]]]]

In [ ]:
"""测试if语句"""
tokens = tokenize(
            'if (x > 0) { print("Positive"); } else { print("Non-positive"); }'
        )
parse(tokens)

['program',
 [['if',
   ['binary', '>', ['id', 'x'], ['num', 0]],
   ['block', [['print', [['str', 'Positive']]]]],
   ['block', [['print', [['str', 'Non-positive']]]]]]]]

In [15]:
"""测试while语句"""
tokens = tokenize("while (x > 0) { x = x - 1; }")
parse(tokens)

['program',
 [['while',
   ['binary', '>', ['id', 'x'], ['num', 0]],
   ['block', [['assign', 'x', ['binary', '-', ['id', 'x'], ['num', 1]]]]]]]]

In [ ]:
"""测试函数定义"""
tokens = tokenize("fun add(a, b) { return a + b; }")
parse(tokens)

['program',
 [['fun_def',
   'add',
   ['a', 'b'],
   ['block', [['return', ['binary', '+', ['id', 'a'], ['id', 'b']]]]]]]]

In [ ]:
"""测试函数调用"""
tokens = tokenize("print(add(2, 3));")
parse(tokens)

['program', [['print', [['call', ['id', 'add'], [['num', 2], ['num', 3]]]]]]]

In [ ]:
"""测试复杂表达式"""
tokens = tokenize("x = (a + b) * (c - d) / 2;")
parse(tokens)

['program',
 [['assign',
   'x',
   ['binary',
    '/',
    ['binary',
     '*',
     ['binary', '+', ['id', 'a'], ['id', 'b']],
     ['binary', '-', ['id', 'c'], ['id', 'd']]],
    ['num', 2]]]]]

In [ ]:
"""测试无效语法"""
parse(tokenize("var x = ;"))

ParserError: cilly parser : 意外的token: ;

# 集成测试

In [17]:
p1 = """
    var pi = 3.1415926;
    
    var area = fun(r) {
        return pi * r * r;
    } ;
    
    print(area(10), area(20));
    
    var abs = fun(x) {
        if(x < 0)
            return -x;
        else
            return x;
    } ;
    
    var sqrt = fun(x) {
        var iter = fun(guess) {
            if( abs(x - guess * guess) < 0.0001 )
                return guess;
            else
                return iter((guess + x / guess) / 2);
        };
        
        return iter(1);
    
    };
    
    print(sqrt(2), sqrt(2) * sqrt(2));
    
    var make_counter = fun(i) {
        return fun() {
            i = i + 1;
            return i;
        };
    };
    
    var c1 = make_counter(0);
    var c2 = make_counter(0);
    
    print(c1(), c1(), c2());
    
    var K = fun(a) {
        return fun(b) {
            return a;
        };
    };
    
    var KI = fun(a) {
        return fun(b) {
            return b;
        };
    };

    var pair = fun(a,b) {
        return fun(f) {
            return f(a)(b);
        };
    };
    
    var first = fun(p) {
        return p(K);
    };
    
    var second = fun(p) {
        return p(KI);
    };
    
    var p = pair(1,2);
    
    print(first(p), second(p));
    
    var fact = fun(n) {
        if(n == 0)
            return 1;
        else
            return n * fact(n-1);
   };
   
   var fact2 = fun(n) {
       var r = 1;
       var i = n;
       
       while(n > 0) {
           r = n * r;
           n = n - 1;
       }
       
       return r;
       
   };
"""

parse(tokenize(p1))

['program',
 [['define', 'pi', ['num', 3.1415926]],
  ['define',
   'area',
   ['fun',
    ['r'],
    ['block',
     [['return',
       ['binary',
        '*',
        ['binary', '*', ['id', 'pi'], ['id', 'r']],
        ['id', 'r']]]]]]],
  ['print',
   [['call', ['id', 'area'], [['num', 10]]],
    ['call', ['id', 'area'], [['num', 20]]]]],
  ['define',
   'abs',
   ['fun',
    ['x'],
    ['block',
     [['if',
       ['binary', '<', ['id', 'x'], ['num', 0]],
       ['return', ['unary', '-', ['id', 'x']]],
       ['return', ['id', 'x']]]]]]],
  ['define',
   'sqrt',
   ['fun',
    ['x'],
    ['block',
     [['define',
       'iter',
       ['fun',
        ['guess'],
        ['block',
         [['if',
           ['binary',
            '<',
            ['call',
             ['id', 'abs'],
             [['binary',
               '-',
               ['id', 'x'],
               ['binary', '*', ['id', 'guess'], ['id', 'guess']]]]],
            ['num', 0.0001]],
           ['return', ['id', '